In [1]:
import numpy as np
import pandas as pd 
import string
import random
import csv
import time
import math
import multiset 
import plotly.express as px
import json 
import pandas as pd 

## Reading Data Files 

In [4]:
# The target file includes the original sequences that encodes the data. 
with open('targetBlawat et al.LDPC.txt') as f:
    reads = f.readlines()
for i in range(0,len(reads)):
    reads[i] = reads[i][0:len(reads[i])-1]
# reads 

In [5]:
np.size(reads)

9608

In [6]:
# The errors_shuffled file include the erroneous copies shuffled. 
with open('errors_shuffled.txt') as f:
    reads_err = f.readlines()
for i in range(0,len(reads_err)):
#     reads_err[i] = reads_err[i][0:len(reads_err[i])-1]
    reads_err[i] = reads_err[i].strip()
# reads_err

In [7]:
np.size(reads_err)

1161706

In [8]:
# evyat.txt include the erroneous copies clustered and labeled. 
with open('evyat.txt') as f:
    reads_cl = f.readlines()
for i in range(0,len(reads_cl)):
    reads_cl[i] = reads_cl[i].strip()
# reads_cl

# Functions


In [9]:
def hash_fun_ind(x,a,w,l): 
    cnt = 0 
    for i in range(0,len(x)):
        if x[i] == a[cnt]:
            cnt += 1  
        else:
            cnt = 0 
        if cnt == w:
            return i-w+1 
    return -1

def hash_fun(x,a,w,l):
    ind = hash_fun_ind(x,a,w,l)
    return x[ind:min(len(x),ind+w+l)]

def ind_st(st):
    #All 3-grams: AAA,AAG,AAC,AAT,AGA,AGC,...,TTA,TTG,TTC,TTT
    # A-0, G-1, C-2, T-3
    # index of CAT = Decimal representaion of (203)
    N_q = {"A":0,"C":1,"G":2,"T":3}
    dec = 0
    for i in range(0,len(st)):
        dec += N_q[st[i]]*(4**(len(st)-i-1))
    return dec 
    
def bin_sig(x,q):
    bs = [0]*(4**q) 
    for i in range(0,len(x)-q+1):
        st = x[i:i+q]
        bs[ind_st(st)] = 1
    return bs 
        
def ham_dis(x,y):
    dis = 0
    for i in range(0,len(x)):
        if(x[i]!= y[i]):
            dis += 1
    return dis 

def rand_perm(w):
    return ''.join(random.choice('ACGT') for _ in range(w))

def merge(x,y,parent):
    parent[x]= min(parent[x],parent[y])
    parent[y]= min(parent[x],parent[y])
    return parent
    
def rep_find(inp,parent):
    temp = inp
    cnt = 0
    while(parent[temp]!=temp and cnt<10):
        cnt += 1
        temp=parent[temp]
    return temp 

def edit_dis(s1, s2):
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

def find_in_arr(el,parent):
    cnt = 0
    for i in range(0,len(parent)):
        if parent[i] == el: 
            cnt += 1 
    return cnt 
        
def readFile(fileName):
        fileObj = open(fileName, "r") #opens the file in read mode
        words = fileObj.read().splitlines() #puts the file into an array
        fileObj.close()
        for i in range(0,len(words)):
            words[i] = int(words[i])
        return words

def read_file_dict(array):
    d = dict(enumerate(array.flatten()))
    return d[0]
    
def display_parent(parent):
    clstr = {}
    for i in range(0,len(parent)):
        clstr[i]=[]
    for i in range(0,len(parent)):
        clstr[rep_find(i,parent)].append(i)
    return clstr

def display_parent_ver2(parent,C_til_prev,parent_prev):
    clstr = C_til_prev
    for i in range(0,len(parent)):
        if(parent[i]!=parent_prev[i]):
#                 print(i,rep_find(i,parent_prev),clstr[rep_find(i,parent_prev)])
            clstr[rep_find(i,parent)].extend(clstr[i])
            clstr[rep_find(i,parent_prev)] = []
    return clstr

In [10]:
q =  3
x = "AACCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA"
y = "AACACGAAATAACACACCACGAAGAGACACGACACAACACAACAGAGCACAACAGAGCACAACAC"
ham_dis(bin_sig(x,q),bin_sig(y,q))

23

In [11]:
q

3

In [12]:
edit_dis(x,y)

28

In [13]:
x = reads_err[1]
y = reads_err[215]
ham_dis(bin_sig(x,q),bin_sig(y,q))

7

In [14]:
edit_dis(x,y)

4

# Clustering Algorithm

In [15]:
def rep_in_C(read,C_reps):
    lower = 0;
    upper = len(C_reps) - 1;
    while (lower <= upper):
        mid = lower + int((upper - lower) / 2)
#         print(upper,mid)
        res = -1
        if (read == (C_reps[mid][0])):
            return C_reps[mid][1]
        if (read > (C_reps[mid][0])):
            lower = mid + 1;
        else:
            upper = mid - 1;
    return -1;

def comp_clstrs(alg_clstr,org_clstr,gamma):
    num_exist = 0
    if(len(alg_clstr)>len(org_clstr)):
#         print(alg_clstr)
        return 0 
    else:
        for i in range(0,len(alg_clstr)):
            flg_exist = 0
            for j in range(0,len(org_clstr)):
                if(reads_err[alg_clstr[i]] == org_clstr[j]):
                    flg_exist = 1
                    num_exist +=1
                    break
            if(flg_exist == 0):
                return 0
        if(num_exist < gamma*len(org_clstr)):
            return 0
            
        return 1 
    
def calc_acrcy(clustering,C_dict,C_reps,gamma):
#     clustering = display_parent(parent)
    acrcy = 0
    for i in range(0,len(clustering)):
        if(len(clustering[i])>=1):
            acrcy = acrcy + comp_clstrs(clustering[i],C_dict[rep_in_C(reads_err[clustering[i][0]],C_reps)],gamma)
    return acrcy

def min_max(val1,val2):
    min_val = min(val1,val2)
    max_val = max(val1,val2)
    return min_val,max_val
    

In [16]:
# (Read, Cluster rep of the cluster to which the read belongs to)
C_reps = []
C_dict = {}
rep = reads_cl[0]
for i in range(1,len(reads_cl)):
    if reads_cl[i] != "":
        if reads_cl[i][0] == "*":
            if(len(C_reps)>0):
                C_dict[rep].pop()
                C_reps.pop()
            rep=reads_cl[i-1]
            C_dict[rep]=[]
        else:
#             print(C_dict)
            C_dict[rep].append(reads_cl[i])
            C_reps.append((reads_cl[i],rep))
C_reps.sort(key=lambda x:x[0])

In [17]:
C_reps[0:5]

[('AAAAAAACCGGAATATCAATCGAGCGAATGGGATACTACATTAAGGGATATCACTTGATCACAAG',
  'AACGAAACCGGAATATCAATCGAGCGAATGGGATACTACATTAAGGGATATCACTTGATCACAAG'),
 ('AAAAAAAGGTGAACTCGAAGACGTTAACGATATGACCCGTCAATTAAATGACGTATACACTACT',
  'AACTAAAGGTGAACTCGAAGATCGTTAACGATATGACCCGTCAATTAAATGACGTATACACTACT'),
 ('AAAAACACACTCAACGATACCTCTAACGGATAATCCGGTCACTAACATAAGAAGCACGCAAT',
  'AACTACAAACACACTCAACGATACTCTAACGGATAATCCGGTCACTAACATAAGAAGCACGCAAT'),
 ('AAAAACACCAATCAGATAAGAGTATAAGAGATAAGAACATCTACGAATCCACGCAATATGACG',
  'AACCAAACACCAATCAGATAAGAGTATAAGAGATAAGAACATCTACGAATCCACGCAATATGACG'),
 ('AAAAACACTTAGAAACATCAAGAGCAGCATATCTACCTCGAGTCCGTCTGCACACAACGTAGC',
  'AACTAAACACTTAGAAACATCAAGAGCAGCATATCAACCTCGAGTCCGTCTGCACACAACGTAGC')]

In [18]:
C_dict[list(C_dict.keys())[0]][0:5]

['AATCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA',
 'AACCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA',
 'AACCTCTAGACACACAACTATTAGTACCTATTACAAACACAACATAAGGTCAAGATAAGCAGAGA',
 'AACCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA',
 'AACCTCTAGACAACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA']

In [19]:
# Precompute all the binary signatures
q = 3
bin_sig_arr  = [0]*(len(reads_err))
for i in range(0,len(reads_err)):
    bin_sig_arr[i]=bin_sig(reads_err[i],q)

In [65]:
reads_err_ind  = [0]*(len(reads_err))
parent = [0]*(len(reads_err))
for i in range(0,len(reads_err)):
    reads_err_ind[i] = (i,reads_err[i]) 
    parent[i] = i

C_til = display_parent(parent)
acrcy_dict1 = {}
acrcy_dict2 = {}
acrcy_dict3 = {}
acrcy_dict4 = {}
acrcy_dict5 = {}

time_itr_dict = {}
# parent_dict = {}
# parent_dict[0] = parent.copy() 

# DESIRED_ACRCY = 0.99
gamma = 0.9
card_C = len(reads)
local_comm = 302

q = 3
th_high = 12
th_low  = 8
r = 6
w = math.ceil(math.log(len(reads_err[0]),4))
l = math.ceil(math.log(len(reads_err),4))

lcl_step = 0


for lcl_step in range(0,local_comm):
# while(acrcy < DESIRED_ACRCY):   
    time_start = time.time()

    hash_select = [0]*(len(reads_err))    
    for i in range(0,len(C_til)):
        if(len(C_til[i])!=0):    
            hash_select[random.choice(C_til[i])]=1
            
    a = rand_perm(w)
    hash_C_til = [0]*(len(reads_err))
    for i in range(0,len(reads_err_ind)):
        if(hash_select[i]==0):
            hash_C_til[i] = (i,"")
        else:
            hash_C_til[i] = (i,hash_fun(reads_err_ind[i][1],a,w,l))    
    hash_C_til.sort(key=lambda x:x[1])

    cnt = 0
    for i in range(0,len(hash_C_til)-1):
        if(hash_C_til[i][1] == ""):
            continue 
        else:
            if hash_C_til[i][1] == hash_C_til[i+1][1]:
#                 if rep_find(i,parent) == rep_find(i+1,parent):
#                     continue 
                x = reads_err[hash_C_til[i][0]]
                y = reads_err[hash_C_til[i+1][0]]
                if((ham_dis(bin_sig_arr[hash_C_til[i][0]],bin_sig_arr[hash_C_til[i+1][0]]) <= th_low) or ((ham_dis(bin_sig_arr[hash_C_til[i][0]],bin_sig_arr[hash_C_til[i+1][0]]) <= th_high) and edit_dis(x,y)<=r)):
                    cnt +=  1                   
                    min_temp,max_temp = min_max(rep_find(hash_C_til[i][0],parent),rep_find(hash_C_til[i+1][0],parent))                    
                    C_til[min_temp].extend(C_til[max_temp])
                    C_til[max_temp] = []        
                    parent[max_temp] = min_temp                    

    if lcl_step%10==1:
        acrcy_dict1[lcl_step+1] = calc_acrcy(C_til,C_dict,C_reps,0.6)/card_C
        acrcy_dict2[lcl_step+1] = calc_acrcy(C_til,C_dict,C_reps,0.7)/card_C
        acrcy_dict3[lcl_step+1] = calc_acrcy(C_til,C_dict,C_reps,0.8)/card_C
        acrcy_dict4[lcl_step+1] = calc_acrcy(C_til,C_dict,C_reps,0.9)/card_C
        acrcy_dict5[lcl_step+1] = calc_acrcy(C_til,C_dict,C_reps,1)/card_C
    
#     parent_dict[lcl_step+1]=parent.copy()
    
    time_itr = time.time() - time_start
    time_itr_dict[lcl_step+1]=time_itr
    
    print(time_itr,lcl_step,cnt)
    
#     lcl_step += 1

17.903571844100952 0 215162
76.39730715751648 1 110813
11.986342668533325 2 103003
11.615623950958252 3 77918
7.375612258911133 4 19030
7.784294605255127 5 29409
8.040971517562866 6 40521
9.359636068344116 7 64207
10.051790475845337 8 82391
8.747482776641846 9 64987
5.719127178192139 10 31950
48.967026710510254 11 100241
6.920154333114624 12 40032
6.181757688522339 13 27316
4.709018230438232 14 18754
1.8206775188446045 15 191
4.563801288604736 16 18885
4.033775091171265 17 14527
3.3507444858551025 18 11335
3.0553882122039795 19 8636
1.8114519119262695 20 413
33.73562240600586 21 10765
2.5763347148895264 22 5141
3.272871494293213 23 8221
1.6652321815490723 24 1725
2.456578254699707 25 5922
1.228353500366211 26 667
2.609807014465332 27 5459
1.3083515167236328 28 321
1.682349681854248 29 1315
2.5762126445770264 30 5138
27.702231645584106 31 5239
1.9320855140686035 32 3362
1.7230048179626465 33 2570
1.0499260425567627 34 392
1.2200706005096436 35 1000
1.2230448722839355 36 1031
0.989762306

In [4]:
# with open('time_itr_dict_12,8,6,300_clstr_dict.json', 'w') as fp:
#     json.dump(time_itr_dict, fp)
# with open('acrcy_dict_12,8,6,300,0.6_clstr_dict.json', 'w') as fp:
#     json.dump(acrcy_dict1, fp)
# with open('acrcy_dict_12,8,6,300,0.7_clstr_dict.json', 'w') as fp:
#     json.dump(acrcy_dict2, fp)
# with open('acrcy_dict_12,8,6,300,0.8_clstr_dict.json', 'w') as fp:
#     json.dump(acrcy_dict3, fp)
# with open('acrcy_dict_12,8,6,300,0.9_clstr_dict.json', 'w') as fp:
#     json.dump(acrcy_dict4, fp)
# with open('acrcy_dict_12,8,6,300,1_clstr_dict.json', 'w') as fp:
#     json.dump(acrcy_dict5, fp)

with open('acrcy_dict_12,8,6,300,0.6_clstr_dict.json') as f:
    acrcy_dict1 = json.load(f)
with open('acrcy_dict_12,8,6,300,0.7_clstr_dict.json') as f:
    acrcy_dict2 = json.load(f)
with open('acrcy_dict_12,8,6,300,0.8_clstr_dict.json') as f:
    acrcy_dict3 = json.load(f)
with open('acrcy_dict_12,8,6,300,0.9_clstr_dict.json') as f:
    acrcy_dict4 = json.load(f)
with open('acrcy_dict_12,8,6,300,1_clstr_dict.json') as f:
    acrcy_dict5 = json.load(f)

In [66]:
total_time = 0
for i in range(1,len(time_itr_dict)+1):
    total_time += time_itr_dict[i]
total_time

1319.4379534721375

In [14]:
keys = acrcy_dict1.keys()
values1 = acrcy_dict1.values()
values2 = acrcy_dict2.values()
values3 = acrcy_dict3.values()
values4 = acrcy_dict4.values()
values5 = acrcy_dict5.values()

df = pd.DataFrame()

df["keys"] = keys
df["0.6"]= values1
df["0.7"]= values2
df["0.8"]= values3
df["0.9"]= values4
df["1.0"]= values5

fig = px.line(df, x=df["keys"], y=['0.6', '0.7', '0.8', '0.9', '1.0'])
fig.show()

In [80]:
keys = time_itr_dict.keys()
values = time_itr_dict.values()

px.line(x=keys,y=values)

In [17]:
#  !git status 

In [18]:
# !git add Clustering-algorithm-single-core-\(multiple-values-of-gamma\).ipynb

In [19]:
# !git add -u && git status && git commit -m "clstr_dict and parent_arr approaches" && git push origin master 